# Martech - Load Data

## Initialise Connection

In [1]:
from astra_db import AstraSession

astra = AstraSession()

In [2]:
# Prepare Statements
#update_acty_cap_stmt = astra.session.prepare("INSERT INTO actycap_day (activity, comm_date, comm_time) VALUES (?, ?, ?)")
#update_profile_cap_stmt = astra.session.prepare("INSERT INTO profilecap_day (customer_id, channel, comm_date, comm_time) VALUES (?, ?, ?, ?)")
#update_prty_cap_stmt = astra.session.prepare("INSERT INTO prtycap_day (category_group, category, comm_date, comm_time) VALUES (?, ?, ?, ?)")
#update_channel_cap_stmt = astra.session.prepare("INSERT INTO channelcap_day (channel, comm_date, comm_time) VALUES (?, ?, ?)")

#insert_communication_stmt = astra.session.prepare("INSERT INTO communications (communication_id, comm_datetime, customer_id, channel, category_group, category, activity_name) VALUES (?, ?, ?, ?, ?, ?, ?)")
insert_communication_stmt = astra.session.prepare("INSERT INTO communication_data (comm_id, comm_datetime, request_date, activity, category, channel, customer_id) VALUES (?, ?, ?, ?, ?, ?, ?)")

## Load Data

### Define Entities

In [3]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity
activity_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

### Define Insert Function

In [4]:
from datetime import datetime, timezone
import random
import uuid


def insert_comm(date):
    # format date and time
    #comm_date = int(date.strftime('%Y%m%d')) # format date as YYYYMMDD
    #comm_time = int(date.strftime('%H%M%S%f')[:9]) # format time as HHMMSSsss
    
    # randomise entities
    comm_id = str(uuid.uuid4())
    comm_datetime = date
    request_date = date
    activity = activity_array[random.randint(0, 4)]
    customer_id = customer_id_array[random.randint(0, 10)]
    channel = channel_array[random.randint(0, 4)]
    #category_group = category_group_array[random.randint(0, 4)]
    category = category_array[random.randint(0, 4)]

    # insert caps
    #astra.session.execute(update_acty_cap_stmt, (activity, comm_date, comm_time))
    #astra.session.execute(update_profile_cap_stmt, (customer_id, channel, comm_date, comm_time))
    #astra.session.execute(update_prty_cap_stmt, (category_group, category, comm_date, comm_time))
    #astra.session.execute(update_channel_cap_stmt, (channel, comm_date, comm_time))

    # insert communication
    astra.session.execute(
            insert_communication_stmt, 
            (comm_id, comm_datetime, request_date, activity, category, channel, customer_id)
        )

### Load many

In [ ]:
for i in range(8):
    date = datetime.now(timezone.utc)
    insert_comm(date)

### Load a day

In [15]:
year = 2025
month = 4
day = 13

# insert data randomly across a day
for i in range(10):
    hour = random.randint(0, 23)
    min = random.randint(0, 59)
    sec = random.randint(0, 59)
    ms = random.randint(0, 999999)
    date = datetime(year, month, day, hour, min, sec, ms, tzinfo=timezone.utc)
    insert_comm(date)

## Clean Up

In [5]:
# TRUNCATE TABLES
astra.session.execute("TRUNCATE communication_data")
astra.session.execute("TRUNCATE events_by_day")

astra.session.execute("TRUNCATE profilecap_month")
astra.session.execute("TRUNCATE actycap_day")
astra.session.execute("TRUNCATE prtycap_day")
astra.session.execute("TRUNCATE channelcap_day")

astra.session.execute("TRUNCATE actycap_week")
astra.session.execute("TRUNCATE prtycap_week")
astra.session.execute("TRUNCATE channelcap_week")

In [16]:
astra.shutdown()